In [1]:
!pip install folium
!pip install wget


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9679 sha256=06a6308150f9df85818f4d8a6d7a1f2381fa5515058c324b7e42aa1b3d6fd92c
  Stored in directory: c:\users\e a p\appdata\local\pip\cache\wheels\01\46\3b\e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import folium
import wget
import pandas as pd

In [4]:
from folium.plugins import MarkerCluster

from folium.plugins import MousePosition

from folium.features import DivIcon

In [5]:
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

100% [................................................................................] 7710 / 7710

In [6]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [7]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [8]:
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [9]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

for index, row in launch_sites_df.iterrows():
    name = row['Launch Site']
    latitude = row['Lat']
    longitude = row['Long']
    coordinate = [latitude, longitude]
    
    circle = folium.Circle(coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(name))
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html="<div style='font-size: 12; color:{};'><b>{}</b></div>". format('#d35400',name)
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

In [10]:
site_map

In [11]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [12]:
marker_cluster = MarkerCluster()

In [13]:
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [14]:
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():

    name = record['Launch Site']
    latitude = record['Lat']
    longitude = record['Long']
    coordinate = [latitude, longitude]
    color = record['marker_color']
    
    marker = folium.map.Marker(
        location=coordinate,
        popup=name,
        icon=folium.Icon(color='white', icon_color=color)
        )
    
    marker_cluster.add_child(marker)

site_map

In [15]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [16]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [17]:
spacex_df[spacex_df['Launch Site'] == 'VAFB SLC-4E']

,Launch Site,Lat,Long,class,marker_color
26,VAFB SLC-4E,34.632834,-120.610745,0,red
27,VAFB SLC-4E,34.632834,-120.610745,0,red
28,VAFB SLC-4E,34.632834,-120.610745,1,green
29,VAFB SLC-4E,34.632834,-120.610745,1,green
30,VAFB SLC-4E,34.632834,-120.610745,1,green
31,VAFB SLC-4E,34.632834,-120.610745,1,green
32,VAFB SLC-4E,34.632834,-120.610745,0,red
33,VAFB SLC-4E,34.632834,-120.610745,0,red
34,VAFB SLC-4E,34.632834,-120.610745,0,red
35,VAFB SLC-4E,34.632834,-120.610745,0,red


In [18]:
launch_site_lat = 34.632834
launch_site_lon = -120.610746
coastline_lat = 34.63543
coastline_lon = -120.62508

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [19]:
coordinate = [34.63543, -120.62508]

distance_marker = folium.map.Marker(
        location = coordinate,
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )

site_map.add_child(distance_marker)

In [20]:
import folium.plugins as plugins

In [21]:
def draw_line(launch_site_lat, launch_site_lon, place_lat, place_lon, place_name):
    launch_coordinate = [launch_site_lat, launch_site_lon]
    place_coordinate = [coastline_lat, coastline_lon]

    marker = folium.map.Marker(
        location=place_coordinate, 
        popup=name,
        tooltip='click',
        icon=folium.Icon(color='red')
    )
    site_map.add_child(marker)

    coordinates=[]
    coordinates.append(launch_coordinate)
    coordinates.append(place_coordinate)
    
    distance = calculate_distance(launch_site_lat, launch_site_lon, place_lat, place_lon)

    text = "{} {:0.2f}km".format(place_name, distance)
    marker = folium.map.Marker(
        location=place_coordinate, 
        tooltip=text,
        icon=plugins.BeautifyIcon(
            number=round(distance,0),
            border_color='#8D208B',
            border_width=2,
            text_color='#8D208B',
            inner_icon_style='margin-top:0px;')
    )
    site_map.add_child(marker)
    
    linetext = "{}{:0.2f} km".format(place_name, distance)
    folium.PolyLine(coordinates, color='#325931', weight=1.5, popup=linetext).add_to(site_map)

In [22]:
launch_site_lat = 34.632834
launch_site_lon = -120.610746
coastline_lat = 34.63543
coastline_lon = -120.62508

draw_line(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon, "Coastline")
site_map

In [23]:
draw_line(launch_site_lat, launch_site_lon, 34.63858, -120.45839, "Lompoc city")

draw_line(launch_site_lat, launch_site_lon, 34.63263, -120.62507, "Santa Barbara Subdivision MT1")

draw_line(launch_site_lat, launch_site_lon, 34.63469, -120.61575, "Agena Way")

site_map